<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## LSTM Bot QA

## Alumna: Maria Fabiana Cid

## Desafío 4

### Datos
El objecto es utilizar datos disponibles del challenge ConvAI2 (Conversational Intelligence Challenge 2) de conversaciones en inglés. Se construirá un BOT para responder a preguntas del usuario (QA).\
[LINK](http://convai.io/data/)

In [ ]:
!pip install --upgrade --no-cache-dir gdown --quiet

In [ ]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [ ]:
import re
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, LSTM, SimpleRNN, Embedding, Input
from sklearn.model_selection import train_test_split


In [ ]:
# Descargar la carpeta de dataset
import os
import gdown
if os.access('data_volunteers.json', os.F_OK) is False:
    url = 'https://drive.google.com/uc?id=1awUxYwImF84MIT5-jCaYAPe2QwSgS1hN&export=download'
    output = 'data_volunteers.json'
    gdown.download(url, output, quiet=False)
else:
    print("El dataset ya se encuentra descargado")

Downloading...
From: https://drive.google.com/uc?id=1awUxYwImF84MIT5-jCaYAPe2QwSgS1hN&export=download
To: /content/data_volunteers.json
100%|██████████| 2.58M/2.58M [00:00<00:00, 56.1MB/s]


In [ ]:
# dataset_file
import json

text_file = "data_volunteers.json"
with open(text_file) as f:
    data = json.load(f) # la variable data será un diccionario



In [ ]:
# Observar los campos disponibles en cada linea del dataset
data[0].keys()

dict_keys(['dialog', 'start_time', 'end_time', 'bot_profile', 'user_profile', 'eval_score', 'profile_match', 'participant1_id', 'participant2_id'])

In [ ]:
chat_in = []
chat_out = []

input_sentences = []
output_sentences = []
output_sentences_inputs = []
max_len = 30

def clean_text(txt):
    txt = txt.lower()
    txt.replace("\'d", " had")
    txt.replace("\'s", " is")
    txt.replace("\'m", " am")
    txt.replace("don't", "do not")
    txt = re.sub(r'\W+', ' ', txt)

    return txt

for line in data:
    for i in range(len(line['dialog'])-1):
        # vamos separando el texto en "preguntas" (chat_in)
        # y "respuestas" (chat_out)
        chat_in = clean_text(line['dialog'][i]['text'])
        chat_out = clean_text(line['dialog'][i+1]['text'])

        if len(chat_in) >= max_len or len(chat_out) >= max_len:
            continue

        input_sentence, output = chat_in, chat_out

        # output sentence (decoder_output) tiene <eos>
        output_sentence = output + ' <eos>'
        # output sentence input (decoder_input) tiene <sos>
        output_sentence_input = '<sos> ' + output

        input_sentences.append(input_sentence)
        output_sentences.append(output_sentence)
        output_sentences_inputs.append(output_sentence_input)

print("Cantidad de rows utilizadas:", len(input_sentences))

Cantidad de rows utilizadas: 6033


In [ ]:
input_sentences[1], output_sentences[1], output_sentences_inputs[1]

('hi how are you ', 'not bad and you  <eos>', '<sos> not bad and you ')

In [ ]:


df = pd.read_json('data_volunteers.json')



### 2 - Preprocesamiento
Realizar el preprocesamiento necesario para obtener:
- word2idx_inputs, max_input_len
- word2idx_outputs, max_out_len, num_words_output
- encoder_input_sequences, decoder_output_sequences, decoder_targets

In [ ]:

# Limpieza simple (minúsculas, quitar caracteres raros)
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9¿?¡!]+", " ", text)
    text = text.strip()
    return text

input_sentences = [clean_text(s) for s in input_sentences]

# Para outputs, agregamos tokens especiales para indicar inicio y fin
output_sentences = ["<start> " + clean_text(s) + " <end>" for s in output_sentences]

#  tokenización

# Tokenizer para inputs
tokenizer_inputs = Tokenizer(filters='', oov_token='<OOV>')
tokenizer_inputs.fit_on_texts(input_sentences)
word2idx_inputs = tokenizer_inputs.word_index  # diccionario palabra->índice
input_sequences = tokenizer_inputs.texts_to_sequences(input_sentences)
max_input_len = max(len(seq) for seq in input_sequences)

# Tokenizer para outputs
tokenizer_outputs = Tokenizer(filters='', oov_token='<OOV>')
tokenizer_outputs.fit_on_texts(output_sentences)
word2idx_outputs = tokenizer_outputs.word_index
output_sequences = tokenizer_outputs.texts_to_sequences(output_sentences)
max_out_len = max(len(seq) for seq in output_sequences)
num_words_output = len(word2idx_outputs) + 1  # +1 porque indexación desde 1

#Padding

encoder_input_sequences = pad_sequences(input_sequences, maxlen=max_input_len, padding='post')
decoder_output_sequences = pad_sequences(output_sequences, maxlen=max_out_len, padding='post')

# Preparar targets del decoder (desplazados un paso)

decoder_targets = np.zeros_like(decoder_output_sequences)
decoder_targets[:, 0:-1] = decoder_output_sequences[:, 1:]



### 3 - Preparar los embeddings
Utilizar los embeddings de Glove o FastText para transformar los tokens de entrada en vectores

In [ ]:
import os
import gdown
import zipfile

#  Descargar FastText embeddings (1M vocabulario, 300d)
if not os.path.exists('wiki-news-300d-1M.vec'):
    print("Descargando FastText embeddings...")
    url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip'
    output = 'wiki-news-300d-1M.vec.zip'
    gdown.download(url, output, quiet=False)

    # Paso 2: Descomprimir
    print("Descomprimiendo archivo...")
    with zipfile.ZipFile(output, 'r') as zip_ref:
        zip_ref.extractall()
    print("Listo. Archivo extraído.")

else:
    print("Archivo FastText ya existe, no se descarga.")


embedding_dim = 300
embeddings_index = {}

print("Cargando FastText embeddings... (tarda unos minutos)")
with open('wiki-news-300d-1M.vec', 'r', encoding='utf-8', errors='ignore') as f:
    next(f)  # saltar línea header
    for line in f:
        values = line.rstrip().split(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print(f"Embeddings FastText cargados: {len(embeddings_index)}")


Descargando FastText embeddings...


Downloading...
From: https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
To: /content/wiki-news-300d-1M.vec.zip
100%|██████████| 682M/682M [00:06<00:00, 113MB/s]


Descomprimiendo archivo...
Listo. Archivo extraído.
Cargando FastText embeddings... (tarda unos minutos)
Embeddings FastText cargados: 999994


In [ ]:

# Crear matriz de embeddings para encoder
num_encoder_tokens = len(word2idx_inputs) + 1
embedding_matrix = np.zeros((num_encoder_tokens, embedding_dim))

for word, i in word2idx_inputs.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.random.normal(scale=0.6, size=(embedding_dim,))

### 4 - Entrenar el modelo
Entrenar un modelo basado en el esquema encoder-decoder utilizando los datos generados en los puntos anteriores. Utilce como referencias los ejemplos vistos en clase.

In [ ]:
# Encoder
encoder_inputs = Input(shape=(max_input_len,), name='encoder_inputs')
encoder_embedding = Embedding(
    input_dim=num_encoder_tokens,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=max_input_len,
    trainable=False,
    mask_zero=False
)(encoder_inputs)

encoder_lstm = LSTM(256, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
num_decoder_tokens = num_words_output
decoder_inputs = Input(shape=(max_out_len,), name='decoder_inputs')

decoder_embedding_layer = Embedding(
    input_dim=num_decoder_tokens,
    output_dim=embedding_dim,
    input_length=max_out_len,
    trainable=True,
    mask_zero=False
)

decoder_embedding = decoder_embedding_layer(decoder_inputs)

decoder_lstm = LSTM(256, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

# Modelo final
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, 12)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 9, 300)    │    538,800 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 12, 300)   │    541,200 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 256),     │    570,368 │ embedding[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, 12, 256), │    570,368 │ embedding_1[0][0… │
│                     │ (None, 256),      │            │ encoder_lstm[0][… │
│                     │ (None, 256)]      │            │ encoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, 12, 1804)  │    463,628 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,684,364 (10.24 MB)

 Trainable params: 2,145,564 (8.18 MB)

 Non-trainable params: 538,800 (2.06 MB)

In [ ]:
# Entrenamiento
model.fit(
    [encoder_input_sequences, decoder_output_sequences],
    decoder_targets[..., np.newaxis],  # <- necesario para sparse_categorical_crossentropy
    batch_size=64,
    epochs=50,
    validation_split=0.2
)

Epoch 1/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.4905 - loss: 3.9303 - val_accuracy: 0.6847 - val_loss: 1.9370
Epoch 2/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.6786 - loss: 1.8232 - val_accuracy: 0.7011 - val_loss: 1.7752
Epoch 3/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7015 - loss: 1.6403 - val_accuracy: 0.7100 - val_loss: 1.6737
Epoch 4/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7326 - loss: 1.5345 - val_accuracy: 0.7332 - val_loss: 1.6020
Epoch 5/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7470 - loss: 1.4378 - val_accuracy: 0.7334 - val_loss: 1.5585
Epoch 6/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7558 - loss: 1.3686 - val_accuracy: 0.7480 - val_loss: 1.4976
Epoch 7/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.7666 - loss: 1.3086 - val_accuracy: 0.7499 - val_loss: 1.4743
Epoch 8/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.7714 - loss: 1.2647 - val_accuracy: 0.7572 - v

### 5 - Inferencia
Experimentar el funcionamiento de su modelo. Recuerde que debe realizar la inferencia de los modelos por separado de encoder y decoder.

In [ ]:
# Modelo encoder para inferencia
encoder_model = Model(encoder_inputs, encoder_states)

# Inputs para el decoder (uno por cada estado + la palabra anterior)
decoder_state_input_h = Input(shape=(256,), name='decoder_state_input_h')
decoder_state_input_c = Input(shape=(256,), name='decoder_state_input_c')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Reusar capa de embedding y LSTM del decoder
decoder_emb_infer = decoder_embedding_layer(decoder_inputs)
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
    decoder_emb_infer, initial_state=decoder_states_inputs)

decoder_states = [state_h_inf, state_c_inf]
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

# Modelo decoder para inferencia
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states
)


In [ ]:
def decode_sequence(input_seq, tokenizer_inputs, tokenizer_outputs, max_input_len, max_out_len):
    # Codificar el input
    input_seq = tokenizer_inputs.texts_to_sequences([clean_text(input_seq)])
    input_seq = pad_sequences(input_seq, maxlen=max_input_len, padding='post')

    # Obtener los estados iniciales del encoder
    states_value = encoder_model.predict(input_seq)

    # Generar <start> token para comenzar
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer_outputs.word_index['<start>']

    # Decodificación paso a paso
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Elegir palabra con mayor probabilidad (greedy search)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_outputs.index_word.get(sampled_token_index, '')

        # Parar si es token de fin o se excede longitud máxima
        if sampled_word in ['<end>', 'eos', ''] or len(decoded_sentence.split()) >= max_out_len:
            stop_condition = True
        else:
            decoded_sentence += sampled_word + ' '

        # Actualizar target_seq y estados para siguiente paso
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()


In [ ]:
input_text = "Hello"
respuesta = decode_sequence(input_text, tokenizer_inputs, tokenizer_outputs, max_input_len, max_out_len)
print("Bot:", respuesta)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Bot: hello how are you


###Análisis de la respuesta:


1. Sobre la respuesta del modelo

Respuesta: "hello how are you"

Es una respuesta sencilla, coherente y natural en inglés, adecuada como saludo y continuación a la entrada "Hello".

Esto indica que el modelo captó el contexto básico y genera frases de forma secuencial con sentido.

2. Sobre el proceso y tiempos

El modelo realizó una predicción paso a paso (decodificación token a token).

En total hizo 6 predicciones (6 pasos) para generar esa respuesta.

El tiempo por paso (~30-38 ms) es rápido y normal para una inferencia en CPU o GPU.

3. Sobre el modelo en general

Parece que la arquitectura seq2seq funciona y está correctamente entrenada para respuestas simples.

El hecho de que responda con frases naturales indica que el modelo aprendió el patrón básico de saludo.

###Resumen:

El modelo está funcionando para inferencia, generando respuestas coherentes en secuencia.

La inferencia paso a paso es estándar para seq2seq con decodificador LSTM.

El resultado sugiere un modelo funcional aunque probablemente simple o entrenado con pocos datos.